In [1]:
# 需要另外下載的module： svglib, Pillow, gpxpy, matplotlib, requests, beautifulsoup4

from tkinter import *
from tkinter.ttk import Separator
import os
import requests
from bs4 import BeautifulSoup
import time # 不要被 ban
from svglib.svglib import svg2rlg #存成 png
from reportlab.graphics import renderPM #存成 png
from PIL import Image, ImageTk #將圖片塞到 GUI
import math  #計算經緯度距離
import gpxpy.parser as parser  #用來讀GPX檔案，並存成經緯度、高度時間資料
import datetime  # 此段計算點間距離、高度差、斜率及每隔多少距離的斜率
import matplotlib.pyplot as plt # 此段繪製等高線圖

file_path = os.getcwd()

win = Tk()
win.title('登山訓練一把罩')
win.geometry('1920x1080')
win.config(bg = '#323232')

#scroll1 = Scrollbar(win)
#scroll1.pack(side = RIGHT, fill = Y)
#scroll2 = Scrollbar(fram, orient = HORIZONTAL)
#scroll2.pack(side = BOTTOM, fill = X)

# 建立標題「登山訓練一把罩」

title_text = Label(text = '登山訓練一把罩', fg = 'white', bg = '#323232', font = '華康粗圓體(P) 22')
title_text.pack(pady = 10 , padx = 10)
sep = Separator(orient = HORIZONTAL)
sep.pack(fill = X , pady = 5 )

#==================================

# 建立總體框架

fram = Frame(win, bg = '#323232')
fram.pack()

farm = Frame(win, bg = '#323232')  # 資料顯示部分
farm.pack()

#=============================================

# 建立第一框架「先幫山取個綽號吧！」

#fram1 = Frame(fram, bg = '#323232')
#fram1.pack()

# 找不到你要的山嗎？

#add_mt = Label(fram1, text = '先幫山取個綽號吧！', fg = 'white', bg = '#323232', font = '標楷體 14')
#add_mt.pack(pady = 9, padx = 5)

#=============================================

# 建立第二框架「新增山名」

#fram2 = Frame(fram, bg = '#323232')
#fram2.pack()

#org_mt = Label(fram2, text = '原本山的名字', fg = 'white', bg = '#323232', font = '標楷體 10')
#org_mt.pack(side = LEFT, pady = 9, padx = 5)
#org_mt_name = Entry(fram2)
#org_mt_name.pack(side = LEFT,  pady = 9, padx = 5)

#new_mt = Label(fram2, text = '新增山的別名', fg = 'white', bg = '#323232', font = '標楷體 10')
#new_mt.pack(side = LEFT,  pady = 9, padx = 5)
#new_mt_name = Entry(fram2)
#new_mt_name.pack(side = LEFT, pady = 9, padx = 5)

# 建立新增暱稱按鈕
        
#btn = Button(fram2, text = '新增暱稱', highlightbackground = '#323232', fg = 'white', bg = '#323232', font = '微軟正黑體 10')
#btn.pack(side = LEFT,  pady = 9, padx = 5)

#==================================

# 建立第三部分「今天想爬哪座山」

fram3 = Frame(fram, bg = '#323232')
fram3.pack()

ask_mt = Label(fram3,text = '今天想爬哪座山？', fg = 'white', bg = '#323232', font = '標楷體 14')
ask_mt.pack(side = LEFT,pady = 5, padx = 5)

#==================================

# 哪座百岳？

fram4 = Frame(fram3, bg = '#323232')
fram4.pack()

ans_mt = Entry(fram3)
ans_mt.pack(side = LEFT, pady = 9, padx = 5)

#找路線
route_mtdict = {'合歡東峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=287', ['合歡東峰步道', '合歡東峰']], '合歡北峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=288', ['合歡北峰步道', '合歡北峰']], '合歡石門山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=413', ['合歡石門山步道', '石門山', '桃園石門山']], '合歡主峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=289', ['合歡主峰步道', '合歡主峰']], '玉山主峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=299', ['玉山主峰步道', '玉山主峰']], '雪山主東峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=431', ['雪山主東峰步道', '雪山主峰', '雪山東峰']], '奇萊南峰步道、南華山步道(奇萊南華) ': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=430', ['奇萊南峰步道、南華山步道(奇萊南華) ', '南華山']], '嘉明湖步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=347', ['嘉明湖步道', '向陽山', '三叉山']], '北大武山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=338', ['北大武山步道', '北大武山']], '合歡西峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=536', ['合歡西峰步道', '合歡北峰', '合歡主峰', '合歡西峰']], '大霸群峰登山步道(大鹿林道線)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=242', ['大霸群峰登山步道(大鹿林道線)', '大霸尖山', '小霸尖山', '加利山', '伊澤山']], '奇萊主山、奇萊北峰步道(奇萊主北)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=542', ['奇萊主山、奇萊北峰步道(奇萊主北)', '奇萊主山', '奇萊北峰']], '桃山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=429', ['桃山登山步道', '桃山']], '武陵四秀登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=540', ['武陵四秀登山步道', '池有山', '桃山', '喀拉業山', '品田山']], '南湖群峰登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=547', ['南湖群峰登山步道', '南湖大山', '南湖大山東峰', '南湖北山', '南湖大山南峰', '巴巴山', '審馬陣山']], '郡大山、望鄉山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=500', ['郡大山、望鄉山登山步道', '郡大山']], '畢祿山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=535', ['畢祿山登山步道', '畢祿山']], '台灣池步道(合歡北峰+小溪營地+北峰名樹)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=690', ['台灣池步道(合歡北峰+小溪營地+北峰名樹)', '合歡北峰']], '羊頭山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=482', ['羊頭山步道', '羊頭山']], '志佳陽大山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=531', ['志佳陽大山登山步道', '志佳陽大山']], '雪山下翠池': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=691', ['雪山下翠池', '雪山主峰', '雪山東峰']], '玉山前五峰登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=591', ['玉山前五峰登山步道', '玉山主峰', '玉山東峰', '玉山北峰', '玉山西峰', '玉山前峰']], '西巒大山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=617', ['西巒大山登山步道', '西巒大山']], '畢羊縱走(畢祿羊頭縱走)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=655', ['畢羊縱走(畢祿羊頭縱走)', '畢祿山', '羊頭山']], '玉山西峰步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1046', ['玉山西峰步道', '玉山西峰']], '玉山群峰線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=621', ['玉山群峰線', '玉山東峰', '玉山主峰', '玉山北峰', '玉山南峰', '南玉山', '玉山西峰', '玉山前峰', '東小南山', '鹿山']], '白姑大山登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=653', ['白姑大山登山步道', '白姑大山']], '閂山、鈴鳴山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=532', ['閂山、鈴鳴山步道', '鈴鳴山', '閂山']], '屏風山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=656', ['屏風山步道', '屏風山']], '北三段(能高安東軍)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=652', ['北三段(能高安東軍)', '能高主山', '能高山南峰', '光頭山', '白石山', '安東軍山']], '奇萊連峰登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=644', ['奇萊連峰登山步道', '奇萊北峰', '奇萊主山', '奇萊南峰', '南華山']], '北一段(南湖、中央尖)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=646', ['北一段(南湖、中央尖)', '審馬陣山', '南湖北山', '馬比杉山', '南湖大山', '巴巴山', '中央尖山']], '南二段': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=632', ['南二段', '八通關山', '大水窟山', '達芬尖山', '塔芬山', '轆轆山', '雲峰', '三叉山', '向陽山', '南雙頭山']], '天巒池下武法奈尾山出將軍廟': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=692', ['天巒池下武法奈尾山出將軍廟', '合歡北峰']], '八大秀(八通關山、大水窟山、秀姑巒山)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=623', ['八大秀(八通關山、大水窟山、秀姑巒山)', '八通關山', '秀姑巒山', '大水窟山']], '合歡西峰下華崗': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1032', ['合歡西峰下華崗', '合歡北峰', '合歡西峰']], '大霸北稜線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=639', ['大霸北稜線', '大霸尖山', '伊澤山', '加利山', '小霸尖山']], '玉山後四峰登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=590', ['玉山後四峰登山步道', '玉山南峰', '東小南山', '南玉山', '鹿山']], '雪山西稜線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=641', ['雪山西稜線', '雪山東峰', '雪山主峰', '中雪山', '大雪山', '頭鷹山', '火石山']], '大小劍(大劍山、小劍山)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=689', ['大小劍(大劍山、小劍山)', '推論山', '佳陽山', '大劍山']], '聖稜線O型縱走(O聖)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=636', ['聖稜線O型縱走(O聖)', '桃山', '喀拉業山', '池有山', '品田山', '雪山北峰', '雪山主峰', '雪山東峰']], '北二段': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=666', ['北二段', '閂山', '鈴鳴山', '無明山', '甘薯峰']], '秀霸線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=637', ['秀霸線', '池有山', '品田山', '大霸尖山', '伊澤山', '加利山']], '奇萊東稜': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=643', ['奇萊東稜', '奇萊北峰', '磐石山', '太魯閣大山', '立霧主山', '帕托魯山']], '南一段': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=630', ['南一段', '庫哈諾辛山', '關山', '海諾南山', '小關山', '卑南主山']], '雪劍線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=640', ['雪劍線', '雪山東峰', '雪山主峰', '大劍山', '佳陽山', '小劍山']], '馬博拉斯橫斷(馬博橫斷)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=625', ['馬博拉斯橫斷(馬博橫斷)', '八通關山', '秀姑巒山', '馬博拉斯山', '駒盆山', '馬西山']], '八通關日治越道線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=627', ['八通關日治越道線', '八通關山']], '新康橫斷出瓦拉米': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=631', ['新康橫斷出瓦拉米', '布拉克桑山', '新康山']], '聖稜線I型縱走(I聖)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=719', ['聖稜線I型縱走(I聖)', '雪山主峰', '雪山北峰', '雪山東峰', '大霸尖山', '伊澤山', '加利山']], '干卓萬群峰登山步道': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=881', ['干卓萬群峰登山步道', '干卓萬山', '牧山', '萬東山西峰', '卓社大山']], '南三段(丹大、東郡橫斷)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=667', ['南三段(丹大、東郡橫斷)', '丹大山', '內嶺爾山', '義西請馬至山', '東郡大山', '無雙山', '東巒大山']], '聖稜線Y型縱走(Y聖)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=710', ['聖稜線Y型縱走(Y聖)', '品田山', '池有山', '桃山', '喀拉業山', '雪山北峰', '雪山主峰', '加利山', '伊澤山', '大霸尖山']], '能高越嶺西段上卡賀爾、能高主峰': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1507', ['能高越嶺西段上卡賀爾、能高主峰', '能高山']], '治茆山連走西巒大山、巒安堂': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=908', ['治茆山連走西巒大山、巒安堂', '西巒大山']], '奇萊北壁下屏風山': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1485', ['奇萊北壁下屏風山', '奇萊北峰', '屏風山']], '淡蘭古道北路（澳底－大里段）': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1707', ['淡蘭古道北路（澳底－大里段）', '遠望坑山', '灣坑頭山']], '合歡西稜': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1618', ['合歡西稜', '合歡主峰']], '中央尖、死亡稜線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1474', ['中央尖、死亡稜線', '中央尖山', '甘薯峰']], '六順山、七彩湖登山步道（丹大進出）': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1668', ['六順山、七彩湖登山步道（丹大進出）', '六順山']], '六順山、七彩湖登山步道(萬榮進、丹大出)': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1665', ['六順山、七彩湖登山步道(萬榮進、丹大出)', '六順山']], '志佳陽雪山線': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1689', ['志佳陽雪山線', '雪山東峰', '雪山主峰', '志佳陽大山']], '小關山林道上小關山': ['https://hiking.biji.co/index.php?q=trail&act=detail&id=1677', ['小關山林道上小關山', '小關山']]}
urlhead = 'https://hiking.biji.co'
def mt_rout():
    userinput = ans_mt.get()
    mtinroute = list() #回傳使用者輸入的山岳被包含在哪幾條步道
    for key,value in route_mtdict.items():
        if userinput in value[1]:
            mtinroute.append(key)
    mt.config(text = mtinroute)
    if len(mtinroute) == 0:
        mt.config(text = 'Oops，找不到你要的山唷，請重新輸入', fg = 'white', bg = '#323232', font = '微軟正黑體 11')

# 建立查詢路線按鈕
        
btn = Button(fram3, text = '找路線', highlightbackground = '#323232', fg = 'white', bg = '#323232', font = '微軟正黑體 10', command = mt_rout)
btn.pack(side = LEFT,pady = 5, padx = 5)

# 顯示該百岳路線

fram_routes = Frame(fram, bg = '#323232')
fram_routes.pack()

mt = Label(fram_routes, text = '', fg = 'white', bg = '#323232', font = '微軟正黑體 11')
mt.pack(side = LEFT,pady = 9, padx = 5)

#==================================

fram5 = Frame(fram, bg = '#323232')
fram5.pack()

# 哪條路線？

ask_rout = Label(fram5, text = '想要走哪條路線？', fg = 'white', bg = '#323232', font = '標楷體 14')
ask_rout.pack(side = LEFT , pady = 9, padx = 5)

#==================================

fram6 = Frame(fram, bg = '#323232')
fram6.pack()

rt = Label(fram6, text = '', fg = 'white', bg = '#323232', font = '微軟正黑體 11')
rt.pack(side = LEFT,pady = 9, padx = 5)

ans_rout = Entry(fram5)
ans_rout.pack(side = LEFT , pady = 9, padx = 5)
to_download_lst = []

    #第二步：依序讀取資料夾內 GPX 檔 for i in list
    #第三步：丟到上面的GPX程式裡面解讀
    #加入，讀下載了的圖檔，丟進來
    #第四步：把讀出來的數據丟到frame呈現
#用距離去取checkpoint的算法

# ===============================
def gpx():
    userinput = ans_rout.get()
    non_rt = ['聖稜線Y型縱走(Y聖)', '治茆山連走西巒大山、巒安堂']
    if userinput in non_rt:
        oops = Label(fram6, text = '恭喜！你找到了沒有GPX的路線！成為這條路的開拓者吧！', fg = 'white', bg = '#323232', font = '微軟正黑體 11')
        oops.pack(pady = 9, padx = 5)
        return
    elif userinput in route_mtdict.keys():
        #此段為製作 list 測試用，gpx_list 出來後刪掉
        def findgpx(userinput):
            def findgpxinf(gpxcell):
                li = 0
                for download in gpxcell.select('ul.row-stack-list.info-list li'):
                    li +=1 
                    if li == 4:
                        gpxnameraw = gpxcell.h3.text
                        gpxname = gpxnameraw.replace('\n','')
                        gpxname = gpxname.replace('/','_') #修改gpx名變成跟下載的檔案一樣
                        a = 0
                        for gpxdownload in gpxcell.select('div.func-list a'):
                            a += 1
                            if a == 2:
                                gpxurl = urlhead + gpxdownload['href']
                                break
                        downloadcell = download.text.split()
                        #0603 將,從數字拿掉 0611改用replace寫
                        downloadnum = int(downloadcell[0].replace(',',''))
                        gpxinf.append([gpxname, gpxurl, downloadnum])
                        break

            page = 1
            gpxinf = list() #裝GPX資訊的list[[GPX名,網址,下載次數],...]
            urlroute = value[0]
            reroute = requests.get(urlroute)
            souproute = BeautifulSoup(reroute.text, 'html.parser')
            try:
                for moregpx in souproute.select('div.more-gpx'):
                    urlmoregpx = urlhead + moregpx.a['href']
                    regpx = requests.get(urlmoregpx)            
                    soupgpx = BeautifulSoup(regpx.text, 'html.parser')
                    for totalgpxcell in soupgpx.select('div.flex.items-baseline.gpx-title span'):
                        celllist = totalgpxcell.text.split()
                        totalgpxnum = int(celllist[0])
                        #print(totalgpxnum)
                    countgpx = 1

                    for gpxcell in soupgpx.select('div.postMeta-feedSummery'):
                        if page-1 == (totalgpxnum//20) and countgpx == totalgpxnum :
                            findgpxinf(gpxcell)
                            break
                        elif countgpx % 20 == 1 and countgpx > 1:
                            break               
                        findgpxinf(gpxcell)
                        countgpx += 1

                if not page-1 == (totalgpxnum//20) and countgpx == totalgpxnum :
                    while True :
                        page += 1
                        urlnextpage = urlmoregpx + '&page=' + str(page)
                        renextpage = requests.get(urlnextpage)            
                        soupnextpage = BeautifulSoup(renextpage.text, 'html.parser')

                        for gpxcell in soupnextpage.select('div.postMeta-feedSummery'):
                            if page-1 == (totalgpxnum//20) and countgpx == totalgpxnum :
                                break
                            elif countgpx % 20 == 1 and countgpx-1 == page*20:
                                break
                            findgpxinf(gpxcell)
                            countgpx += 1                
                        if page-1 == (totalgpxnum//20) and countgpx == totalgpxnum :
                            findgpxinf(gpxcell)
                            break
                        elif countgpx % 20 == 1 and countgpx-1 == page*20:
                            continue
            #避免沒有'更多gpx'按鈕的路線bug
            except:        
                for nomoregpx in souproute.select('h3.list-title'):
                    try:
                        urlnomoregpx = urlhead + nomoregpx.a['href']
                        renomoregpx = requests.get(urlnomoregpx)            
                        soupnomoregpx = BeautifulSoup(renomoregpx.text, 'html.parser')
                        for gpxpage in soupnomoregpx.select('div.primary-info'):
                            gpxname = gpxpage.h1.text
                            gpxname = gpxname.replace('/','_')
                            gpxurl = urlnomoregpx.replace('gpx_detail','download_gpx')
                            span = 0
                            for gpxdownload in gpxpage.select('div.flex.items-center.metrics-list span'):
                                span += 1
                                if span == 4:
                                    downloadcell = gpxdownload.text.split()
                                    downloadnum = int(downloadcell[0].replace(',',''))
                                    gpxinf.append([gpxname, gpxurl, downloadnum])                                       
                    except:
                        break
            #print(gpxinf)    
            gpxinfsorted = sorted(gpxinf, reverse = True, key = lambda d:d[2])
            #print(gpxinfsorted)
            #0611 把to_download_lst換到這
            for g in gpxinfsorted[0:3]:
                to_download_lst.append(g)
            #print(to_download_lst)

        userinput = ans_rout.get()
        for key,value in route_mtdict.items():
            if userinput == value[1][0] :
                findgpx(userinput)

        #此段為下載，測試用，gpx_list 出來後刪掉
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
            'Cookie':'_ga=GA1.2.793198669.1589982449; __auc=4219b9b71723256d6edb54ab191; _fbp=fb.1.1589982452087.605399371; AviviD_canv=0; AviviD_uuid=bf489202-2afb-4dc7-9121-56457ef808f0; webuserid=26fe2e65-5bc8-2224-26d0-8ce6aaf35e97; __gads=ID=c1af8b4c905cdcd6:T=1590146410:S=ALNI_Mbsg2PqKkHnkl39HtES8yG8iBIa5Q; AviviD_refresh_uuid_status=2; __eruid=b3f5386f-1e30-c581-1112-679b252d7c8a; AviviD_waterfall_status=2; _gid=GA1.2.1855126731.1590814335; AviviD_show_sub=1; RELOGIN=y; __erEvntUid=942222; PHPSESSID=s0b29gmkv0i58q80coko2joiu7; __asc=0f964d9d17278c5e2b15ddd19f8; __lastv=942222; page_view=3; USERNAME=Train+Mt; AVATAR=https%3A%2F%2Fgraph.facebook.com%2F115529540180480%2Fpicture%3Ftype%3Dlarge; EID=MViLczHYAtK2P1RftIwMIQ%3D%3D',
        }

        session = requests.Session()
        naming = 2

        if not os.path.exists(userinput):
            os.makedirs(userinput) 

        for gpxinf in to_download_lst:
            if not os.path.exists(os.path.join(userinput,gpxinf[0])+".gpx"):
                with open(os.path.join(userinput,gpxinf[0])+".gpx","w", encoding='utf-8') as f:
                    r = session.get(gpxinf[1], headers=headers)
                    f.write(r.text)
                    r.close()
            else:
                with open(os.path.join(userinput,gpxinf[0])+str(naming)+'.gpx',"w", encoding='utf-8') as f:
                    gpxinf[0] += str(naming)
                    r = session.get(gpxinf[1], headers=headers)
                    f.write(r.text)
                    naming += 1
                    r.close()

        #此段計算經緯度距離
        def distance(x1, y1, x2, y2):

            R = 6373.0
            lat1 = math.radians(x1)
            lon1 = math.radians(y1)
            lat2 = math.radians(x2)
            lon2 = math.radians(y2)
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2 #Haversine公式
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

            return (R * c)

        # ===============================

        #此段用來讀GPX檔案，並存成經緯度、高度時間資料
        for p in to_download_lst:#裝GPX資訊的list[[GPX名,網址,下載次數],...]
            try :
             # 開啟gpx檔
                gpx_file = open(os.path.join(file_path,userinput,p[0]+'.gpx'), encoding="utf-8")
                gpx_parser = parser.GPXParser( gpx_file )  # 抓取gpx檔中的trk, trkseg與trkpt標籤
                gpx = gpx_parser.parse() # 讀取trk, trkseg與trkpt標籤

                #trk = gpx.tracks  # gpx檔trk中的內容，資料形式為list
                #trkseg = trk[0].segments # 抓取 trk 中的 trkseg標籤內容，資料形式為list
                #trkpt = trkseg[0].points # 抓取 trkseg中的 trkpt 內容，資料形式為list
                # trk[0], trkseg[0], trkpt[0] 的資料形式分別為(gpxpy.gpx.)GPXTrack, GPXTrackSegment, 與GPXTrackPoint

                latitude_x = []     #未整理座標緯度
                longitude_x = []    #未整理座標經度
                elevation_x = []    #未整理座標高度
                time_x = []         #未整理座標時間

                for trk in gpx.tracks:
                    for trkseg in trk.segments:
                        for trkpt in trkseg.points :
                            latitude_x.append(trkpt.latitude)
                            longitude_x.append(trkpt.longitude)
                            elevation_x.append(trkpt.elevation)
                            time_x.append(trkpt.time)

                latitude = []     # 排序後的座標緯度
                longitude = []    # 排序後的座標經度
                elevation = []    # 排序後的座標高度
                time = []         # 排序後的座標時間    

                no_time_mark = 0    # 如果GPX沒有時間，則後面資料不要顯示時間

                if None not in time_x:
                    no_time_mark += 1
                    tuples_lst = list(zip(latitude_x,longitude_x,elevation_x,time_x))
                    tuples = sorted(tuples_lst, key = lambda lst:lst[3])
                    for i in tuples:
                        latitude.append(i[0])
                        longitude.append(i[1])
                        elevation.append(i[2])
                        time.append(i[3])
                else:
                    latitude = latitude_x
                    longitude = longitude_x
                    elevation = elevation_x
                    time = time_x

                # ===============================
                # 此段計算點間距離、高度差、斜率及每隔多少距離的斜率

                dist = [] #點與點之間的距離差
                elev = [] #點與點之間的高度差
                slope = [] #點與點之間的斜率

                for i in range(len(latitude)-1):  # 算斜率
                    x = distance(latitude[i], longitude[i], latitude[i+1], longitude[i+1])*1000
                    y = elevation[i] - elevation[i+1]
                    dist.append(x)
                    elev.append(y)
                    if x != 0:
                        slope.append(y/x)

                # ===============================

                # 此段計算總里程、垂直上下高度

                slant_dist = []  #點與點之間的斜距，單位公尺

                for i in range(len(dist)):  # 總實際距離
                    slant_dist.append(math.sqrt(dist[i]**2 + elev[i]**2))

                all_dist = round(sum(dist)/1000)  # 總平面距離

                uphill, downhill = 0, 0  # 垂直上升／下降高度

                for i in elev:
                    if i > 0:
                        uphill += i
                    else:
                        downhill += i

                # ===============================

                #此段算每 x 平面距離的斜率與換算角度

                checkpoint = 0  # 記錄點 1
                nextpoint = 1   # 記錄點 2
                dis_slope = []  # 記錄每50公尺的斜率（坡度）
                up_count_lst = []    # >50公尺的連續陡上坡
                down_count_lst = []  # >50公尺的連續陡下坡
                up_count = 0
                down_count = 0

                while nextpoint < len(latitude):  # 計算 dis_slope
                    if distance(latitude[checkpoint], longitude[checkpoint], latitude[nextpoint], longitude[nextpoint])*1000 < 50:  # 如果距離小於50公尺
                        nextpoint += 1
                    else:
                        x1, y1 = latitude[checkpoint], longitude[checkpoint]  # 起始點經緯度
                        x2, y2 = latitude[nextpoint], longitude[nextpoint]  # 相隔 50公尺（以上）的經緯度
                        dis_slope.append((elevation[nextpoint]-elevation[checkpoint])/(distance(x1, y1, x2, y2)*1000)) #套入經緯度距離函式，以高度除以距離（公尺／公尺）
                        checkpoint = nextpoint
                        if dis_slope[-1] > 0.37: # 如果角度大於20.3度
                            up_count += 50  # 就把把距離 x 算進連續陡上
                        elif dis_slope[-1] < 0.37 and up_count != 0:
                            up_count_lst.append(up_count)
                            up_count = 0
                        if dis_slope[-1] < -0.46:  # 如果角度大於 24.7度
                            down_count += 50
                        elif dis_slope[-1] > -0.46 and down_count != 0:
                            down_count_lst.append(down_count)
                            down_count = 0

                if up_count_lst == []:
                    up_count_lst.append(0)
                if down_count_lst == []:
                    down_count_lst.append(0)

                degree = []  # 記錄距離間隔斜率（坡度）換算的角度

                for i in dis_slope:
                    angle = math.degrees(math.atan(i))
                    degree.append(angle)

                _0to10, _10to15 , _15up= 0,0,0  # 計算坡角度比率

                for i in degree:
                    if 0 < abs(i) <= 10:
                        _0to10 += 1
                    if 10< abs(i) <= 15:
                        _10to15 += 1
                    if abs(i) > 15:
                        _15up += 1

                # ===============================
                # 此段繪製等高線圖

                dist_= [0] #距離list
                s = 0
                for i in dist:
                    s += i/1000 #把距離m換算成km
                    dist_.append(s)

                elev_ = [] #高度list
                for i in elevation:
                    elev_.append(i)

                #畫高度距離表
                plt.plot(dist_,elev_)
                plt.ylabel('Altitude(m)')
                plt.xlabel('Distance(km)')
                plt.savefig(p[0]+'.svg')
                plt.show()

                drawing = svg2rlg(p[0]+'.svg')
                renderPM.drawToFile(drawing, p[0]+'.png', fmt='PNG')

        # 第二部分：呈現GPX的frame

                z = Frame(farm, bg = '#323232')
                z.pack(side = LEFT,  pady = 15, padx = 40)

                photo = Image.open(p[0]+'.png')  # 開啟圖像位置
                photo.thumbnail((240,180))  # 依照比例重設大小，因為是照比例，所以他的長取決於高，主要看高多少，不要讓下面的字被吃掉就好
                photo.save(p[0]+'new.png')  # 後面輸入檔名，我不知道預設會存在哪，我的是存在我存py檔的地方
                new = ImageTk.PhotoImage(file = p[0]+'new.png') #開啟新存的圖像
                z.new = new
                lb = Label(z, image = z.new)  #將新存圖像插入Label
                lb.pack()

                naaame = Label(z, text = p[0],  fg = 'white', bg = '#323232', font = '微軟正黑體 11')
                naaame.pack(pady = 5)
                
                                
                teeest = "、".join(route_mtdict[userinput][1][1:])

                mtinside = Label(z, text = '這條路線有'+teeest,  fg = 'white', bg = '#323232', font = '微軟正黑體 11', wraplength = 360)
                mtinside.pack(pady = 5)

                difficulty = ['Ultra','Hard','Medium','Easy','Rookie']
                climb_time = [datetime.timedelta(days=3, seconds=33536, microseconds=222000), datetime.timedelta(days=1, seconds=36319), datetime.timedelta(seconds=47937), datetime.timedelta(seconds=20533), datetime.timedelta(seconds=0)]
                highest = [3745, 3467, 3356, 3105, 0]
                ave_hight = [3159, 2844, 2687, 2359, 0]
                flat_rt = [58, 33, 18, 8, 0]
                real_rt = [65, 38, 20, 8, 0]
                ten_po = [63, 54, 48, 35, 0]
                between_po = [18, 15, 13, 9, 0]
                fifteen_po = [36, 29, 24, 10, 0]
                go_up = [6371, 3606, 2199, 830, 0]
                go_down = [6783, 3706, 2229, 830, 0]
                steep_up_all = [2800, 1900, 1100, 250, 0]
                steep_up = [450, 300, 200, 100, 0]
                steep_down_all = [350, 250 , 150 , 50 , 0]
                steep_down = [1750, 1150, 650, 50, 0]

                endurance = [] # 耐力類
                legs_train = [] # 腿部訓練

                if no_time_mark != 0:
                    for i in range(len(difficulty)):
                        if time[-1]-time[0] >= climb_time[i]:
                            info1 = Label(z, text = '攀爬時間'+str(time[-1]-time[0])+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                            info1.pack()
                            endurance.append(difficulty[i])
                            break
                else:
                    info1 = Label(z, text = '該路線GPX無時間資料' , fg = 'white', bg = '#323232', font = '標楷體 11')
                    info1.pack()

                for i in range(len(difficulty)):
                    if round(max(elevation)) >= highest[i]:
                        info2 = Label(z, text = '最高點'+str(round(max(elevation)))+'m'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info2.pack()
                        endurance.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if round(sum(elevation)/len(elevation)) >= ave_hight[i]:
                        info3 = Label(z, text = '平均高度'+str(round(sum(elevation)/len(elevation)))+'m'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info3.pack()
                        endurance.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if all_dist >= flat_rt[i]:
                        info4 = Label(z, text = '平面路程'+str(all_dist)+'km'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info4.pack()  # 因為跟實際路程重複，所以不加難度
                        break
                for i in range(len(difficulty)):
                    if round(sum(slant_dist)/1000) >= real_rt[i]:
                        info5 = Label(z, text = '實際路程'+str(round(sum(slant_dist)/1000))+'km'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info5.pack()
                        endurance.append(difficulty[i])
                        break         
                for i in range(len(difficulty)):  # 因為緩坡越多越簡單，所以難度順序顛倒，用[-i]
                    if int(_0to10/len(degree)*100) >= ten_po[i]:
                        info6 = Label(z, text = '0~10度坡'+str(int(_0to10/len(degree)*100))+'%'+'  *難度: '+difficulty[-i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info6.pack()
                        endurance.append(difficulty[i])  # 因為緩坡越多耐力也越重要，所以用[i]
                        break
                for i in range(len(difficulty)):
                    if round(sum(slant_dist)/1000) >= between_po[i]:
                        info7 = Label(z, text = '10~15度坡'+str(int(_10to15/len(degree)*100))+'%'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info7.pack()
                        legs_train.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if int(_15up/len(degree)*100) >= fifteen_po[i]:
                        info8 = Label(z, text = '15度以上坡'+str(int(_15up/len(degree)*100))+'%'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info8.pack()
                        legs_train.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if round(uphill) >= go_up[i]:
                        info9 = Label(z, text = '總垂直上升'+str(round(uphill))+'公尺'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info9.pack() # 跟總垂直下降重複所以不加
                        break
                for i in range(len(difficulty)):
                    if round(abs(downhill)) >= go_down[i]:
                        info10 = Label(z, text = '總垂直下降'+str(round(abs(downhill)))+'公尺'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info10.pack()
                        endurance.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if sum(up_count_lst) >= steep_up_all[i]:
                        info11 = Label(z, text = '陡升總距離'+str(sum(up_count_lst))+'m'+'(陡升：>20.3度)'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info11.pack()
                        legs_train.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if max(up_count_lst) >= steep_up[i]:
                        info12 = Label(z, text = '最長陡升距離'+str(max(up_count_lst))+'m'+'  *難度: '+difficulty[i] , fg = 'white', bg = '#323232', font = '標楷體 11')
                        info12.pack()
                        legs_train.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if sum(down_count_lst) >= steep_down_all[i]:
                        info13 = Label(z, text = '陡降總距離'+str(sum(down_count_lst))+'m'+'(陡降：>24.7度)'+'  *難度: '+difficulty[i], fg = 'white', bg = '#323232', font = '標楷體 11')
                        info13.pack()
                        legs_train.append(difficulty[i])
                        break
                for i in range(len(difficulty)):
                    if max(down_count_lst) >= steep_down[i]:
                        info14 = Label(z, text = '最長陡降距離'+str(max(down_count_lst))+'m'+'  *難度: '+difficulty[i], fg = 'white', bg = '#323232', font = '標楷體 11')
                        info14.pack()
                        legs_train.append(difficulty[i])
                        break

                running_po = [7,5,3,1,0]
                cross_jump = [90, 75, 60, 45, 30]
                mt_climber = [90, 75, 60, 45, 30]
                poppy = [11, 9, 7, 5, 3]

                info15 = Label(z, text = '建議訓練菜單（每週至少兩次）', fg = 'red', bg = 'yellow', font = '微軟正黑體 11')
                info15.pack(pady = 9)

                #跑步訓練
                for i in difficulty:
                    if i == max(endurance, key = endurance.count):
                        info17 = Label(z, text = '跑 '+str(running_po[difficulty.index(i)])+' 度坡3000公尺', fg = 'yellow', bg = '#323030', font = '微軟正黑體 11')
                        info17.pack()
                #登山訓練
                for i in difficulty:
                    if i == max(endurance, key = endurance.count):
                        info18 = Label(z, text = '登山式 '+str(mt_climber[difficulty.index(i)])+' 秒（四組）', fg = 'yellow', bg = '#323030', font = '微軟正黑體 11')
                        info18.pack()
                #波比跳
                for i in difficulty:
                    if i == max(endurance, key = endurance.count):
                        info19 = Label(z, text = '波比跳 '+str(poppy[difficulty.index(i)])+' 組（12下／組）', fg = 'yellow', bg = '#323030', font = '微軟正黑體 11')
                        info19.pack()            
                #弓箭步交互蹲跳訓練
                for i in difficulty:
                    if i == max(legs_train, key = legs_train.count):
                        info20 = Label(z, text = '弓箭步交互蹲跳 '+str(cross_jump[difficulty.index(i)])+' 秒（四組）', fg = 'yellow', bg = '#323030', font = '微軟正黑體 11')
                        info20.pack()
                # 高腳杯訓練
                if legs_train[1] == 'Ultra' or legs_train[1] == 'Hard':
                    info21 = Label(z, text = '高腳杯深蹲 12 RM（四組）' , fg = 'yellow', bg = '#323030', font = '微軟正黑體 11')
                    info21.pack()  # 15度坡一定比例以上才有此訓練
                else:  # 要加不然排版跑掉
                    info21 = Label(z, text = '' , fg = 'white', bg = '#323030', font = '微軟正黑體 11')
                    info21.pack()
            except :
                continue
            
        fram.destroy()
    else:
        rt.config(text = '找不到這條路線唷，請依上方顯示路線輸入', fg = 'white', bg = '#323232', font = '微軟正黑體 11')
    #except FileNotFoundError:
      #  rt.config(text = '找不到這條路線唷，請依上方顯示路線輸入', fg = 'white', bg = '#323232', font = '微軟正黑體 11')
    #except UnboundLocalError:
      #  rt.config(text = '找不到這條路線唷，請依上方顯示路線輸入', fg = 'white', bg = '#323232', font = '微軟正黑體 11')   
    
# 建立查詢路線按鈕

btn = Button(fram5, text = '找路線資訊', highlightbackground = '#323232', fg = 'white', bg = '#323232', font = '微軟正黑體 10',command = gpx)
btn.pack(side = LEFT , pady = 5, padx = 5)

win.mainloop()

In [ ]:
合歡北峰步道